In [4]:
import pickle as pickle
import os
import pandas as pd
import numpy as np
import random
import torch
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig, AutoModelForSequenceClassification, AutoConfig
from transformers import AdamW, get_constant_schedule, get_constant_schedule_with_warmup, get_cosine_schedule_with_warmup
from load_data import *
import argparse
from importlib import import_module
from pathlib import Path
import glob
import re
import neptune
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import inference

In [5]:
# model_type = 'Bert'
model_type = 'XLMRoberta'
# MODEL_NAME = 'bert-base-multilingual-cased'
MODEL_NAME = 'xlm-roberta-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model_module = getattr(import_module("transformers"), f"{model_type}" + "ForSequenceClassification")
model = model_module.from_pretrained(MODEL_NAME, num_labels=42)
encoded1 = tokenizer('이순신은 조선 중기의 무신이다. ')
encoded2 = tokenizer('나는 조선 중기의 무신이다. ')
# model


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [10]:
print(len(tokenizer.vocab.items()))
sorted(tokenizer.vocab.items(), key = lambda x: x[1])[:10]

250002


[('<s>', 0),
 ('<pad>', 1),
 ('</s>', 2),
 ('<unk>', 3),
 (',', 4),
 ('.', 5),
 ('▁', 6),
 ('s', 7),
 ('▁de', 8),
 ('-', 9)]

In [12]:
special_tokens_dict = {'additional_special_tokens': ["<E1-PERSON>","</E1-PERSON>","<E2-PERSON>","</E2-PERSON>"]}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

# special_token_key = ["start_entity1_person","end_entity1_person","start_entity2_person","end_entity2_person"]
# special_token_value = ["<E1-PERSON>","</E1-PERSON>","<E2-PERSON>","</E2-PERSON>"]
# special_tokens_dict = {k:v for k,v in zip(special_token_key,special_token_value)}
# tokenizer.add_special_tokens(special_tokens_dict)
print(len(tokenizer.vocab.items()))


250006


In [13]:
num_added_toks

4

In [3]:
encoded1

{'input_ids': [0, 1504, 46364, 5301, 697, 107852, 5432, 127808, 8143, 5301, 5769, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [36]:
[k for k in tokenizer.get_vocab().keys() if '이명박' in k]

[]

In [17]:
print(tokenizer.decode(encoded1['input_ids']))
print(tokenizer.decode(encoded2['input_ids']))

<s> 이순신은 조선 중기의 무신이다.</s>
<s> 나는 조선 중기의 무신이다.</s>


In [10]:
# import transformers
# transformers.AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-mecab-wp-small-discriminator",num_labels=42)

Some weights of the model checkpoint at monologg/koelectra-mecab-wp-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-mecab-wp-small-discriminator and are newly

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

In [37]:
model_type = 'Bert'
MODEL_NAME = 'bert-base-multilingual-cased'
from transformers import AutoTokenizer, AutoModelForPreTraining
  
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-mecab-wp-small-discriminator")
encoded = tokenizer(['이순신은 조선 중기의 무신이다.','이순신은 조선 중기의 무신이다.'])
[tokenizer.decode(s) for s in encoded['input_ids']]

['[CLS] 이순신은 조선 중기의 무신이다. [SEP]', '[CLS] 이순신은 조선 중기의 무신이다. [SEP]']

In [44]:
[k for k in tokenizer.get_vocab().keys() if '선수' in k]

['선수이다', '선수']

In [17]:
model_type

'Bert'

In [20]:
model_module = getattr(import_module("transformers"), f"{model_type}" + "ForSequenceClassification")
# model = AutoModelForPreTraining.from_pretrained('/opt/ml/code/kfold_results/bert-base-multilingual-cased4/0fold/checkpoint-850')
model = model_module.from_pretrained('/opt/ml/code/kfold_results/bert-base-multilingual-cased4/0fold/checkpoint-850')
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen